In [1]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.5 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 24.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [2]:
!pip install requests

In [3]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00


In [4]:
!pip install -q spacy[transformers] torch torchvision torchaudio transformers
!python -m spacy download en_core_web_trf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.3/716.3 kB 11.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 5.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 20.8 MB/s eta 0:00:0000:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import torch
from datasets import load_dataset, Dataset, concatenate_datasets, DatasetDict

from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer, pipeline, BitsAndBytesConfig, EarlyStoppingCallback
from concurrent.futures import ProcessPoolExecutor
from transformers import MarianMTModel, MarianTokenizer
from peft import LoraConfig, get_peft_model

import os
import gc
from sklearn.utils import resample
import numpy as np
import pandas as pd
import json
import random
import time
import requests
from concurrent.futures import ThreadPoolExecutor
from collections import Counter, defaultdict

In [6]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


**1. Loading Dataset**

In [7]:
#--- load data from github----
github_url = "https://raw.githubusercontent.com/jialeCharloote/econ_chatbot/main/Dataset/balanced_news.json"

json_filename = "merged_news.json"
response = requests.get(github_url)

if response.status_code == 200:
    with open(json_filename, "w", encoding="utf-8") as f:
        f.write(response.text)
    print(f"File '{json_filename}' downloaded successfully.")
else:
    print("Failed to download the file. Check the URL or permissions.")

with open(json_filename, "r", encoding="utf-8") as f:
    data = json.load(f)

news_data =  Dataset.from_list(data)
del response, data
news_data

File 'merged_news.json' downloaded successfully.


Dataset({
    features: ['title', 'ticker', 'description', 'sentiment', 'sentiment_reasoning'],
    num_rows: 15977
})

**2. Dataset restructure**

In [8]:
def process_news_data(example):
    """
    Process news data and generate Prompt-Response format for each ticker.
    """
    title = example["title"]
    description = example["description"]
    sentiment_map = {'negative': "Bearish", 'positive': "Bullish", 'neutral':'Neutral'}  # change the label
    label_map = {"Bearish":0, "Bullish":1,'Neutral':2}
    ticker = example["ticker"]
    new_sentiment = sentiment_map[example["sentiment"]]
    sentiment_reasoning = example["sentiment_reasoning"]

    prompt = f"""You are a financial analyst in a leading hedge fund. 
Analyze the sentiment of the following financial news for the given stock ticker step by step.

Title: "{title}"
Summary: "{description}"
Stock Ticker: {ticker}

Step 1: Identify key financial terms and their implications.
Step 2: Determine whether the news suggests market optimism, pessimism, or neutrality for this specific stock.
Step 3: Based on your analysis, classify the sentiment into one of the following categories:
- "Bullish": If the news suggests confidence, growth, or positive impact on this stock.
- "Bearish": If the news suggests decline, risks, or negative impact on this stock.
- "Neutral": If the news is ambiguous or does not convey strong sentiment.

Finally, **return only** the final result in valid JSON format, with the structure:
{{
  "ticker": "{ticker}",
  "sentiment": "Bullish" | "Bearish" | "Neutral",
  "sentiment_reasoning": "Provide a brief explanation of the sentiment analysis."
}}

Do not include any extra text or explanations outside the JSON.
### Response:
(Return a valid JSON format only. Do not repeat the prompt.)
"""
    response = json.dumps({
            "ticker": ticker,
            "sentiment": new_sentiment,
            "sentiment_reasoning": sentiment_reasoning
        })

    return [{"prompt": prompt, "response": response, "labels":label_map[new_sentiment], "ground_truth_sentiment":new_sentiment}]

# **Convert `news_data` into multiple samples corresponding to `prompt-response`**
flattened_data = []
for example in news_data:
    processed = process_news_data(example)
    if processed:
        flattened_data.extend(processed)

news_dataset = Dataset.from_list(flattened_data)

del flattened_data, processed, example, news_data

In [9]:
from collections import Counter
from datasets import concatenate_datasets

counter = Counter(news_dataset["ground_truth_sentiment"])
reduction_ratio = 1
sample_sizes = {label: int(count * reduction_ratio) for label, count in counter.items()}

def sample_filter(example, label, sample_size):
    return example["ground_truth_sentiment"] == label and random.random() < (sample_size / counter[label])

filtered_datasets = []
for label, size in sample_sizes.items():
    filtered_dataset = news_dataset.filter(lambda example: sample_filter(example, label, size))
    filtered_datasets.append(filtered_dataset)

reduced_dataset = concatenate_datasets(filtered_datasets)
reduced_dataset = reduced_dataset.shuffle(seed=42)

print(Counter(reduced_dataset["ground_truth_sentiment"]))

del counter, reduction_ratio, sample_sizes, filtered_datasets, label, size, news_dataset

Filter:   0%|          | 0/15977 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15977 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15977 [00:00<?, ? examples/s]

Counter({'Bullish': 8890, 'Neutral': 4755, 'Bearish': 2332})


In [10]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame(reduced_dataset)
train_df, test_df = train_test_split(
    df, 
    test_size=0.2,  
    stratify=df["ground_truth_sentiment"],  
    random_state=42
)

train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

print("Train set distribution:", Counter(train_dataset["ground_truth_sentiment"]))
print("Test set distribution:", Counter(test_dataset["ground_truth_sentiment"]))
del df, train_df, test_df

Train set distribution: Counter({'Bullish': 7112, 'Neutral': 3804, 'Bearish': 1865})
Test set distribution: Counter({'Bullish': 1778, 'Neutral': 951, 'Bearish': 467})


In [11]:
def concat_prompt_response(example):
    return {"text": example["prompt"] + example["response"]}

train_dataset = train_dataset.map(concat_prompt_response)
test_dataset = test_dataset.map(concat_prompt_response)

Map:   0%|          | 0/12781 [00:00<?, ? examples/s]

Map:   0%|          | 0/3196 [00:00<?, ? examples/s]

**3. Fine tune deepseek-1.5B from Hugging Face**

In [12]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

210

In [ ]:
from huggingface_hub import login
import getpass

hugging_token = getpass.getpass("Enter your Hugging Face Token: ")
login(token=hugging_token)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="fp4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    quantization_config=bnb_config, 
    torch_dtype=torch.float16,
    device_map="auto"
)

model.config.use_cache = False
model.config.sliding_window = None

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [15]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")
    
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

train_dataset = train_dataset.map(tokenize_function, 
                                    batched=True, num_proc=4, 
                                    remove_columns=["prompt", "response", "text"])
test_dataset = test_dataset.map(tokenize_function, 
                                    batched=True, num_proc=4, 
                                    remove_columns=["prompt", "response", "text"])

Map (num_proc=4):   0%|          | 0/12781 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3196 [00:00<?, ? examples/s]

In [16]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

89

In [17]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

training_args = TrainingArguments(
    output_dir="/kaggle/working/finetuned-ds",
    run_name="finetuned-ds-run-1",  
    optim="adamw_torch_fused",
    lr_scheduler_type="cosine_with_restarts",
    learning_rate=4e-5,
    gradient_checkpointing=False, 
    
    num_train_epochs=3,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=6, 
    warmup_ratio=0.1,
    
    weight_decay=0.05,
    logging_dir="./logs",
    logging_steps=100,
    dataloader_pin_memory=True, 
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    
    label_names=["labels"],

    max_grad_norm=5.0,
    fp16=True,  
    dataloader_num_workers=4,  
    push_to_hub=False,
    report_to="none"  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.760300,0.754751
2,0.729400,0.723185
3,0.722000,0.718721


TrainOutput(global_step=6393, training_loss=0.8476288078089713, metrics={'train_runtime': 12194.6296, 'train_samples_per_second': 3.144, 'train_steps_per_second': 0.524, 'total_flos': 1.8196197469323264e+17, 'train_loss': 0.8476288078089713, 'epoch': 3.0})

**4. Save model and training history**

In [18]:
trainer.save_model("/kaggle/working/finetuned-Distillds")
tokenizer.save_pretrained("/kaggle/working/finetuned-Distillds")
print("✅ Training completed. Sucessfully save model")

✅ Training completed. Sucessfully save model


In [19]:
import shutil

# download as zip file
shutil.make_archive("finetuned-Distillds", 'zip', "./finetuned-Distillds")
print("✅ finetuned-Distillds.zip is available to download")

✅ finetuned-Distillds.zip is available to download


In [21]:
loghistory.to_csv("/kaggle/working/loghistory.csv")